# Preprocessing
Putting the images into a folder structure that can be used by keras flow_from_directory

In [10]:
import pandas as pd
import os
from PIL import Image

In [11]:
paths = [
    "calc_case_description_test_set.csv",
    "calc_case_description_train_set.csv",
    "mass_case_description_test_set.csv",
    "mass_case_description_train_set.csv",
]

In [12]:
df = pd.read_csv(f"data/{paths[0]}")
df.head()

,patient_id,breast density,left or right breast,image view,abnormality id,abnormality type,calc type,calc distribution,assessment,pathology,subtlety,image file path,cropped image file path,ROI mask file path
0,P_00038,2,LEFT,CC,1,calcification,PUNCTATE-PLEOMORPHIC,CLUSTERED,4,BENIGN,2,Calc-Test_P_00038_LEFT_CC/1.3.6.1.4.1.9590.100...,Calc-Test_P_00038_LEFT_CC_1/1.3.6.1.4.1.9590.1...,Calc-Test_P_00038_LEFT_CC_1/1.3.6.1.4.1.9590.1...
1,P_00038,2,LEFT,MLO,1,calcification,PUNCTATE-PLEOMORPHIC,CLUSTERED,4,BENIGN,2,Calc-Test_P_00038_LEFT_MLO/1.3.6.1.4.1.9590.10...,Calc-Test_P_00038_LEFT_MLO_1/1.3.6.1.4.1.9590....,Calc-Test_P_00038_LEFT_MLO_1/1.3.6.1.4.1.9590....
2,P_00038,2,RIGHT,CC,1,calcification,VASCULAR,NaN,2,BENIGN_WITHOUT_CALLBACK,5,Calc-Test_P_00038_RIGHT_CC/1.3.6.1.4.1.9590.10...,Calc-Test_P_00038_RIGHT_CC_1/1.3.6.1.4.1.9590....,Calc-Test_P_00038_RIGHT_CC_1/1.3.6.1.4.1.9590....
3,P_00038,2,RIGHT,CC,2,calcification,VASCULAR,NaN,2,BENIGN_WITHOUT_CALLBACK,5,Calc-Test_P_00038_RIGHT_CC/1.3.6.1.4.1.9590.10...,Calc-Test_P_00038_RIGHT_CC_2/1.3.6.1.4.1.9590....,Calc-Test_P_00038_RIGHT_CC_2/1.3.6.1.4.1.9590....
4,P_00038,2,RIGHT,MLO,1,calcification,VASCULAR,NaN,2,BENIGN_WITHOUT_CALLBACK,5,Calc-Test_P_00038_RIGHT_MLO/1.3.6.1.4.1.9590.1...,Calc-Test_P_00038_RIGHT_MLO_1/1.3.6.1.4.1.9590...,Calc-Test_P_00038_RIGHT_MLO_1/1.3.6.1.4.1.9590...


In [13]:
image_types = ["full", "cropped"]
classes = ["benign", "malignant"]

# For each row process the data
for path in paths:
    df = pd.read_csv(f"data/{path}")

    # What dataset is it part of (train or test)
    data_split = path.split("_")[3]
    # Is it mass or calc
    data_set = path.split("_")[0]
    
    # Verify that the folder exists
    for image_type in image_types:
        # Destination for the images
        dest_folder = f"data/{image_type}/{data_set}/{data_split}"

        # Create the folders if they don't exist
        for class_ in classes:
            class_path = f"{dest_folder}/{class_}"
            if not os.path.exists(class_path):
                os.makedirs(class_path)

        # For each row process the data
        for index, row in df.iterrows():

            # Get the image id
            if image_type == "full":
                image_id = row['image file path'].split("/")[2]
            else:
                image_id = row['cropped image file path'].split("/")[2]
            
            image_path = 'archive/jpeg/' + image_id

            # Extract the class
            image_class = row['pathology'].split("_")[0].lower()
            
            try:
                # Only copy the image with the lowest resolution
                images = os.listdir(image_path)
                # Get the resolution of each image
                resolutions = []
                for image in images:
                    loaded_image = Image.open(f"{image_path}/{image}")
                    resolutions.append(loaded_image.size[0] * loaded_image.size[1])
                # Get the lowest resolution file
                lowest_resolution_index = resolutions.index(min(resolutions))
                file = images[lowest_resolution_index]
                os.system(f'cp {image_path}/{file} {dest_folder}/{image_class}/{image_id}_{file}')

            except:
                # If the image doesn't exist, skip it and move on
                print(f"Error: {image_path}")


Error: archive/jpeg/1.3.6.1.4.1.9590.100.1.2.348822970413183698610798947061334416506
